# Test Data Maker

This is a simple application to manually label test data for our usage. Running below will show a text and ask if the given sentence contains the information on target symptom or not. It will automatically start off from where we left off until all descriptions have been assessed. 

In [129]:
import pandas as pd
import joblib
import numpy as np
from math import isnan
from IPython.display import clear_output

### Processing dataset
Run this only once.

In [3]:
## transcription data
#mt = pd.read_csv('data/medical_transcription_samples.csv', index_col = 0)

In [146]:
## get a list of descriptions
#descriptions = pd.concat([mt['description'], mt['transcription']])

In [14]:
## get a list of target symptoms
#symptoms = pd.read_json('data/symptoms.json', orient = 'table')
#symptoms = symptoms['name']

In [148]:
## creating full dataframe
#df = pd.concat([descriptions,pd.DataFrame(columns=list(symptoms) + ['complete'])])
#df.rename(columns = {0: 'description'}, inplace = True)
#df.reset_index(inplace = True)
#df = df.drop('index', axis = 1)

In [149]:
## saving the dataframe
#joblib.dump(df, 'data/test_df')

['data/test_df']

### Labeling App

In [150]:
df = joblib.load('data/test_df')

In [167]:
def labeling_app(df):
    # find incomplete rows
    inds = list(df[df.complete != 1].index)
    
    # randomize the rownumber
    np.random.shuffle(inds)
    ncol = df.shape[1]
    cinds = list(range(1, ncol-1))
    colnames = list(df.columns)
    
    # initialize the last seen variable
    i, j, last_seen = 0, 0, None
    
    while i < len(inds):
        row = inds[i]
        
        # randomize the columns
        np.random.shuffle(cinds)
        
        # show the description
        print(df.iloc[row, 0], end = '\n=====\n')
        print('''Does the above description describe following symptom? (1: yes, 0: no, b: back to previous, x: exit, s: skip, p: pass this description)''')
        
        while j < len(cinds):
            col = cinds[j]
            
            # find incomplete columns

            if isnan(df.iloc[row, col]):    
                ans = input(colnames[col])
                
                # if prompted to finish, save the current dataframe and return the dataframe
                if ans == 'x': 
                    joblib.dump(df, 'data/test_df')
                    return df
                
                # if prompted to go back, bring up the last option again
                elif ans == 'b': 
                    if last_seen: 
                        i = last_seen[0]
                        j = last_seen[1]
                    else: 
                        print('No previous entry available.')
                
                # if prompted to pass the description, move on to the next description
                elif ans == 'p': 
                    last_seen = [i, j]
                    i += 1
                    break
                
                # if prompted to skip, move to the next symptom
                elif ans == 's': 
                    last_seen = [i, j]
                    j += 1
                    
                # if yes or no, update the last seen
                # and update the dataframe 
                elif ans in ['0', '1']: 
                    df.iloc[row, col] = int(ans)
                    last_seen = [i, j]
                    j += 1
                else: 
                    print('')
        if df.iloc[row, :-1].isnull().sum() == 0: 
            df.loc[row, 'complete'] = 1
        i += 1
        clear_output(wait=True)    

In [ ]:
#labeling_app(df)